In [4]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
from openpyxl import Workbook
from datetime import datetime
import os
import requests
import pandas as pd

In [6]:
institutions_uri = 'https://sisu-api.sisu.mec.gov.br/api/v1/oferta/instituicoes'
institution_offers_uri = 'https://sisu-api.sisu.mec.gov.br/api/v1/oferta/instituicao/{co_ies}'
institution_offer_selected_uri = 'https://sisu-api.sisu.mec.gov.br/api/v1/oferta/{co_oferta}/selecionados'

In [7]:
def get_institutions_data():
  response = requests.get(institutions_uri);
  return response.json()

def get_institution_offers_data(institution):
  response = requests.get(institution_offers_uri.replace("{co_ies}", institution["co_ies"]))
  return dict(list(response.json().items())[1:])

def get_institution_offer_selected_data(institution_offer):
  response = requests.get(institution_offer_selected_uri.replace("{co_oferta}", institution_offer["co_oferta"]))
  return response.json()

In [8]:
def append_row(ws, institution, institution_offer, institution_offer_selected):
  if ws.max_row <= 1:
    ws.append(list(institution) + list(institution_offer) + list(institution_offer_selected))
    ws.append(list(institution.values()) + list(institution_offer.values()) + list(institution_offer_selected.values()))

In [14]:
def run():
  current_datetime = datetime.now()

  institutions_data = get_institutions_data()
  data = pd.DataFrame()
  
  for i, institution in enumerate(institutions_data, start=1):
    print(f"Processing institution {i}/{len(institutions_data)}")
    institution_offers_data = get_institution_offers_data(institution)
    
    for j, (_, institution_offer) in enumerate(institution_offers_data.items(), start=1):
      print(f"  Processing offer {j}/{len(institution_offers_data)}")
      institution_offer_selected_data = get_institution_offer_selected_data(institution_offer)
      
      for institution_offer_selected in institution_offer_selected_data:
        inst_df = pd.DataFrame([institution])
        offer_df = pd.DataFrame([institution_offer])
        selected_df = pd.DataFrame([institution_offer_selected])
        
        row_data = pd.concat([inst_df, offer_df, selected_df], axis=1)
        data = pd.concat([data, row_data])
  
  path = "out";
  if not os.path.exists(path):
    os.makedirs(path)

  data.to_csv(f"out/SISUv{current_datetime.year}.csv", index=False)

In [13]:
run()

Processing institution 1/127
  Processing offer 1/24
  Processing offer 2/24
  Processing offer 3/24
  Processing offer 4/24
  Processing offer 5/24
  Processing offer 6/24
  Processing offer 7/24
  Processing offer 8/24
  Processing offer 9/24
  Processing offer 10/24
  Processing offer 11/24
  Processing offer 12/24
  Processing offer 13/24
  Processing offer 14/24
  Processing offer 15/24
  Processing offer 16/24
  Processing offer 17/24
  Processing offer 18/24
  Processing offer 19/24
  Processing offer 20/24
  Processing offer 21/24
  Processing offer 22/24
  Processing offer 23/24
  Processing offer 24/24
